# 데이터 불러오기
필요한 패키지를 임포트하고 데이터를 불러와줍니다.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')  #경고메세지 무시하기

import pandas as pd

train_data = pd.read_csv("/content/data/train.csv")
sample_submission = pd.read_csv("/content/data/sample_submission.csv")

train_data.head()

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,1,10,2392,Ex,3,968,Ex,2392,2392,Ex,2,2003,2003,2003,386250
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
2,3,5,900,TA,1,288,TA,864,900,TA,1,1967,1967,1967,123000
3,4,5,1174,TA,2,576,Gd,680,680,TA,1,1900,2006,2000,135000
4,5,7,1958,Gd,3,936,Gd,1026,1026,Gd,2,2005,2005,2005,250000


출력된 데이터의 모양을 살펴보면 **총 15개의 열(column)**이 존재하는 것을 확인할 수 있습니다.

15 개 중 **2개**는 **id** 와 **target** 열 입니다.

데이터를 자세히 살펴보는 것은 EDA 글을 통해 더 자세히 배워봅시다.

여기에서는 **분석 방법**에 초점을 맞추겠습니다.

데이터를 살펴보면 데이터의 모양이 조금씩 다릅니다.

Gr Liv Area, Garage Cars 과 같은 데이터들은 수치를 나타내는 데이터로 **수치형(numerical) 데이터**라고 합니다.

반면, Kitchen Qual, Bsmt Qual 와 같은 데이터는 수치가 아닌 종류를 나타내죠. 이러한 데이터를 **범주형(categorical) 데이터**라고 합니다.

범주형 데이터의 라벨이 숫자가 아닌 **문자열의 형식으로 저장**되어 있다면 데이터 분석 과정에서 **인코딩(encoding)이 필요**합니다.

## 결측치 확인

결측치(NA: Not Available)란 값이 누락된 데이터를 말합니다.

보다 정확한 분석을 하기 위해서는 데이터의 결측치를 확인하고 적절히 처리해주어야 합니다.

이번 데이터에 결측치가 있나 확인해볼까요?

In [2]:
def check_missing_col(dataframe):
    missing_col = []  #결측치를 가지는 열을 담는 list
    counted_missing_col = 0 #결측치를 가지는 열의 개수
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna()) #열에서 null을 확인한다
        is_missing = True if missing_values >= 1 else False #하나라도 결측치가 존재한다면
        if is_missing:
            counted_missing_col += 1  #결측치를 가지는 열의 개수(counted_missing_col)을 하나 증가시킨다.
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            missing_col.append([col, dataframe[col].dtype]) #list에 열(column)을 추가한다.
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train_data)

결측치가 존재하지 않습니다


### 라벨 인코딩
라벨인코더를 이용해 라벨인코딩을 진행해주려 합니다.

**라벨인코딩**이란, **범주형 변수**를 **숫자로 인코딩**해주는 동시에 컴퓨터가 이들이 각각 다른 범주인 것을 인식하도록 각 값을 **0, 1, 2 등의 값으로 구별**하여 인코딩하는 기법을 말합니다.

라벨 인코딩 함수를 작성해 직접 변수들을 인코딩하겠습니다.

In [3]:
#라벨 인코딩을 하기 전 불필요한 열을 제거하고, 범주형 데이터 열과 수치형 데이터 열을 나누어줍니다. 
train_x = train_data.drop(["id","target"],axis=1) #train_data에서 필요없는 id와 target(=y)를 제거(drop메서드 사용)
train_y = train_data.target #target특징을 train_data의 y에 담는다

num_features = ['Overall Qual', 'Gr Liv Area', 'Garage Cars', #수치형 데이터
       'Garage Area', 'Total Bsmt SF', '1st Flr SF',
       'Full Bath', 'Year Built', 'Year Remod/Add',
       'Garage Yr Blt']

cat_features = ["Exter Qual", "Kitchen Qual","Bsmt Qual"] #범주형 데이터

from sklearn.preprocessing import LabelEncoder  #labelencoder를 불러온다

le = LabelEncoder() #le라는 변수에 labelencoder를 생성

for feature in cat_features:  #범주형 데이터 라벨 인코딩
    train_x[feature] = le.fit_transform(train_x[feature])
    
    #le.fit(X_train)
    #train_x[feature] = le.transform(train_x[feature])의 과정과 동일하다.

### 정규화 
수치형 데이터들을 정규화 시켜줍니다.

머신러닝 과정에서 모델은 데이터의 특성(feature)들을 추출해 학습을 진행합니다.

하지만 학습을 하는 과정에서 데이터의 값이 너무 크거나, 분산이 너무 크면 학습 과정에 악영향을 끼칠 수 있습니다. 

따라서 정규화를 통해 데이터 값의 크기를 줄이고 분산을 줄여 모델이 데이터를 이상하게 해석하는 것을 방지합니다.

이번 베이스라인에서는 min-max 정규화를 이용해 봅시다.

min-max 정규화는 수치형 데이터의 값을 0~1 사이의 값으로 변환해줍니다.

min-max 정규화의 수식은 아래와 같습니다.

X' = (X - MIN) / (MAX-MIN)

In [4]:
#정규화를 위한 패키지를 불러옵니다.
from sklearn.preprocessing import MinMaxScaler

#수치형 데이터를 정규화 시켜줍니다.
scaler = MinMaxScaler() #MinMaxScaler 객체 생성
train_x[num_features] = scaler.fit_transform(train_x[num_features]) #수치형 데이터에 대해 정규화

### 모델 설계 & 학습

#### GridSearchCV 를 사용한 파라미터 최적화

이제 **데이터를 전처리하는 과정**이 끝났습니다. 이제 데이터 분석을 위해 **모델을 설계하는 단계**를 진행해야 합니다.

우선 사용할 **모델을 선택**하여야 합니다. 사용할 모델은 데이터의 특성, task 의 특성에 알맞게 선택해야 합니다.

모델을 선택했다면 **모델의 파라미터를 최적화**하는 과정을 거쳐야 합니다. 

파라미터란 모델의 설정을 말합니다. 예를 들어 Linear Regression 모델의 경우 y 절편을 0으로 설정할 수 있습니다.

파라미터 최적화를 하는 기본적인 방법은 여러 파라미터들을 실험하는 방법입니다.

sklearn 은 GridSearchCV 메소드를 제공합니다.

우선 **선택한 모델과 비교하고 싶은 파라미터들을 설정**해 줍니다. 

그 다음 학습 데이터를 나누어줍니다. 예를 들어 학습 데이터를 5개의 배치로 나누어 주었다고 가정 해보겠습니다.

5개로 나누어진 데이터 중, 4개의 데이터로 학습을 진행합니다. 그 다음 나머지 1개의 데이터 배치로 성능을 검증합니다.

따라서 5번의 학습과 검증이 이루어질 것 입니다. 이때 최상의 성능을 출력하는 파라미터들이 모델의 파라미터로 선택되는 것 입니다.

데이터를 n개의 배치로 나누고, 비교하고자 하는 파라미터들의 조합이 m개라면 총 n * m 번의 학습과 검증이 진행되는 것 입니다.

따라서 학습 시간이 오래 걸리고, 그에 맞는 컴퓨팅 자원이 필요합니다.

이번 베이스라인에서는 GridSearchCV 기법의 개념을 소개하는 방향으로 작성되었습니다.

따라서 앙상블 기법에 사용된 세부적인 모델들의 세부적인 개념들을 추후 베이스라인들에서 소개하겠습니다.

In [5]:
#GridSearchCV 를 사용할 모델들을 호출합니다.
from sklearn.ensemble import RandomForestRegressor  #랜덤포레스트 모듈
from sklearn.ensemble import GradientBoostingRegressor  #그레이디언트 부스팅 모듈
from sklearn.ensemble import ExtraTreesRegressor  #익스트림 랜덤트리 모듈

from sklearn.model_selection import GridSearchCV  #GridSearchCV: 최적의 하이퍼파라미터를 알려준다

#모델들을 할당할 리스트를 만들어줍니다.
estimators = []

#estimators 리스트에 모델들을 추가해줍니다.
rf = RandomForestRegressor()  #랜덤포레스트 생성
estimators.append(rf)

gbr = GradientBoostingRegressor() #그레이디언트 부스팅 생성
estimators.append(gbr)

etr = ExtraTreesRegressor() #익스트림 랜덤트리 생성
estimators.append(etr)

#모들의 파라미터들을 할당할 리스트를 만들어줍니다.
params = []

# params 리스트에 성능을 비교하고자하는 파라미터들 추가해줍니다.
#랜덤포레스트의 인자
params_rf = {'n_estimators' : [90, 100, 110, 120],  #[90, 100, 110, 120] 중 최적의 n_estimators인자 결정
            'min_samples_split' : [1,2,3,4]}  #[1, 2, 3, 4] 중 최적의 min_samples_split인자 결정
params.append(params_rf)

#그레이디언트 부스팅의 인자
params_gbr = {'loss' : ['huber', 'quantile'],
             'learning_rate':[0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15], #최적의 learning_rate인자 결정
             'n_estimators':[60,70,80,90,100,110,120,130,140,150]}  #최적의 n_estimators인자 결정
params.append(params_gbr)

#익스트림 랜덤트리의 인자
params_etr = {'n_estimators' : [50,60,70,80,90,100,110,120,130,140,150]}  #최적의 n_estimators인자 결정
params.append(params_etr)

In [7]:
#GridSearchCV 를 이용해 모델들을 최적화시켜줍니다.
from tqdm.auto import tqdm  #프로그램의 진행상황을 확인할 수 있는 모듈
def gridSearchCV(models,params):
    best_models=[]
    for i in tqdm(range(0,len(models))):
        model_grid = GridSearchCV(models[i], params[i],n_jobs = -1, verbose=1, cv=5)
        model_grid.fit(train_x,train_y) #모델 훈련
        best_models.append(model_grid.best_estimator_)
    return best_models

best_model_list = gridSearchCV(estimators,params)

  0%|          | 0/3 [00:00<?, ?it/s]

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Fitting 5 folds for each of 220 candidates, totalling 1100 fits
Fitting 5 folds for each of 11 candidates, totalling 55 fits


In [8]:
#GridSerachCV 를 통해 최적화된 모델들을 확인합니다.
best_model_list

[RandomForestRegressor(n_estimators=110),
 GradientBoostingRegressor(learning_rate=0.08, loss='huber', n_estimators=140),
 ExtraTreesRegressor(n_estimators=70)]

#### 앙상블 모델 학습

위 코드에서는 3개의 모델을 선택하여 GridSearchCV 를 통해 각 모델들의 파라미터들을 최적화해주었습니다.

그 다음에는 3개의 모델을 모두 활용하여 하나의 모델을 설계해 보겠습니다.

앙살블 기법은 "집단지성" 으로 부터 아이디어를 얻은 알고리즘입니다.
 
모델 하나의 예측 보다는 여러 모델들의 예측을 종합하여 도출한 예측이 더 정확할 때가 많습니다. 

Regression 의 경우 각 모델들이 개별로 예측한 값의 평균 값을 도출하는 방식으로 여러 모델들을 조합해줍니다.

sklearn 패키지의 VotingRegressor 메소드를 통해 앙상블 모델을 구현할 수 있습니다.

GridSearchCV 를 통해 최적의 파라미터들을 찾은 모델들 VotingRegressor 객체에 할당한 후, 학습을 진행합니다.

이번 베이스라인에서는 앙상블 기법의 개념을 소개하는 방향으로 작성되었습니다.

따라서 앙상블 기법에 사용된 세부적인 모델들의 세부적인 개념들을 추후 베이스라인들에서 소개하겠습니다.

In [9]:
#GridSearchCV 를 통해 최적화된 모델들을 사용합니다.
best_models = [
    ('rf', RandomForestRegressor(n_estimators=110)),
    ('GBR', GradientBoostingRegressor(learning_rate=0.08, loss='huber', n_estimators=140)),
    ('ET', ExtraTreesRegressor(n_estimators=70))
]

#앙상블 기법을 위한 패키지를 불러옵니다.
from sklearn.ensemble import VotingRegressor  #개별예측을 평균하여 최종예측을 형성한다.

#앙상블 모델을 학습시켜줍니다.
voting_rg = VotingRegressor(estimators=best_models)
voting_rg.fit(train_x,train_y)  #앙상블한 모델에 훈련

VotingRegressor(estimators=[('rf', RandomForestRegressor(n_estimators=110)),
                            ('GBR',
                             GradientBoostingRegressor(learning_rate=0.08,
                                                       loss='huber',
                                                       n_estimators=140)),
                            ('ET', ExtraTreesRegressor(n_estimators=70))])

### 추론

학습 데이터에 했던 전처리 과정(라벨인코딩, 정규화)를 테스트 데이터에도 해줍니다.

단, 학습 데이터에 학습된 인코더와 정규화 파라미터를 사용해 주어야 합니다. 

(테스트 데이터를 학습 시킨 후 인코딩과 정규화를 진행하면 data leakage 에 해당됩니다!)

In [10]:
test_data = pd.read_csv("/content/data/test.csv")

test_x = test_data.drop("id",axis=1)  #id column 제거

for feature in cat_features:
    test_x[feature] = le.transform(test_x[feature]) #fit 없이 transform만 하는 이유: 새로운 mean값과 variance값을 얻게 test_data가 학습을 하게되기 때문에
    
test_x[num_features] = scaler.transform(test_x[num_features])

In [11]:
#학습된 모델을 통해 test 데이터를 추론합니다.
predictions = voting_rg.predict(test_x)

sample_submission.target = predictions
sample_submission.to_csv("submission.csv",index = False)